# Import & Install Dependencies

In [1]:
# import gc
import numpy as np
import pandas as pd
import datetime as dt
#
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import RobustScaler, LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, log_loss
#
import tensorflow as tf
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras import layers
from sklearn.metrics import mean_absolute_percentage_error
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns', None)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [2]:
pip install flaml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.5/260.5 kB 8.2 MB/s eta 0:00:00


# Import data

In [3]:
train = pd.read_csv('../dataset/train.csv, low_memory=False)
test = pd.read_csv('../dataset/test.csv')
train_target_score = pd.read_csv('../dataset/train_target_and_scores.csv')

In [4]:
train.head()

,id,target,home_team_name,away_team_name,match_date,league_name,league_id,is_cup,home_team_coach_id,away_team_coach_id,home_team_history_match_date_1,home_team_history_match_date_2,home_team_history_match_date_3,home_team_history_match_date_4,home_team_history_match_date_5,home_team_history_match_date_6,home_team_history_match_date_7,home_team_history_match_date_8,home_team_history_match_date_9,home_team_history_match_date_10,home_team_history_is_play_home_1,home_team_history_is_play_home_2,home_team_history_is_play_home_3,home_team_history_is_play_home_4,home_team_history_is_play_home_5,home_team_history_is_play_home_6,home_team_history_is_play_home_7,home_team_history_is_play_home_8,home_team_history_is_play_home_9,home_team_history_is_play_home_10,home_team_history_is_cup_1,home_team_history_is_cup_2,home_team_history_is_cup_3,home_team_history_is_cup_4,home_team_history_is_cup_5,home_team_history_is_cup_6,home_team_history_is_cup_7,home_team_history_is_cup_8,home_team_history_is_cup_9,home_team_history_is_cup_10,home_team_history_goal_1,home_team_history_goal_2,home_team_history_goal_3,home_team_history_goal_4,home_team_history_goal_5,home_team_history_goal_6,home_team_history_goal_7,home_team_history_goal_8,home_team_history_goal_9,home_team_history_goal_10,home_team_history_opponent_goal_1,home_team_history_opponent_goal_2,home_team_history_opponent_goal_3,home_team_history_opponent_goal_4,home_team_history_opponent_goal_5,home_team_history_opponent_goal_6,home_team_history_opponent_goal_7,home_team_history_opponent_goal_8,home_team_history_opponent_goal_9,home_team_history_opponent_goal_10,home_team_history_rating_1,home_team_history_rating_2,home_team_history_rating_3,home_team_history_rating_4,home_team_history_rating_5,home_team_history_rating_6,home_team_history_rating_7,home_team_history_rating_8,home_team_history_rating_9,home_team_history_rating_10,home_team_history_opponent_rating_1,home_team_history_opponent_rating_2,home_team_history_opponent_rating_3,home_team_history_opponent_rating_4,home_team_history_opponent_rating_5,home_team_history_opponent_rating_6,home_team_history_opponent_rating_7,home_team_history_opponent_rating_8,home_team_history_opponent_rating_9,home_team_history_opponent_rating_10,home_team_history_coach_1,home_team_history_coach_2,home_team_history_coach_3,home_team_history_coach_4,home_team_history_coach_5,home_team_history_coach_6,home_team_history_coach_7,home_team_history_coach_8,home_team_history_coach_9,home_team_history_coach_10,home_team_history_league_id_1,home_team_history_league_id_2,home_team_history_league_id_3,home_team_history_league_id_4,home_team_history_league_id_5,home_team_history_league_id_6,home_team_history_league_id_7,home_team_history_league_id_8,home_team_history_league_id_9,home_team_history_league_id_10,away_team_history_match_date_1,away_team_history_match_date_2,away_team_history_match_date_3,away_team_history_match_date_4,away_team_history_match_date_5,away_team_history_match_date_6,away_team_history_match_date_7,away_team_history_match_date_8,away_team_history_match_date_9,away_team_history_match_date_10,away_team_history_is_play_home_1,away_team_history_is_play_home_2,away_team_history_is_play_home_3,away_team_history_is_play_home_4,away_team_history_is_play_home_5,away_team_history_is_play_home_6,away_team_history_is_play_home_7,away_team_history_is_play_home_8,away_team_history_is_play_home_9,away_team_history_is_play_home_10,away_team_history_is_cup_1,away_team_history_is_cup_2,away_team_history_is_cup_3,away_team_history_is_cup_4,away_team_history_is_cup_5,away_team_history_is_cup_6,away_team_history_is_cup_7,away_team_history_is_cup_8,away_team_history_is_cup_9,away_team_history_is_cup_10,away_team_history_goal_1,away_team_history_goal_2,away_team_history_goal_3,away_team_history_goal_4,away_team_history_goal_5,away_team_history_goal_6,away_team_history_goal_7,away_team_history_goal_8,away_team_history_goal_9,away_team_history_goal_10,away_team_history_

# Set up GPU

In [5]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

# USE MULTIPLE GPUS
if os.environ["CUDA_VISIBLE_DEVICES"].count(',') == 0:
    gpu_strategy = tf.distribute.get_strategy()
    print('single strategy')
else:
    gpu_strategy = tf.distribute.MirroredStrategy()
    print('multiple strategy')

single strategy


# FLAML AutoML

## Categorical features with missing values

In [6]:
# categorical features with missing values
categorical_nan = [feature for feature in train.columns if train[feature].isna().sum()>0 and train[feature].dtypes=='O']
print(categorical_nan)

['home_team_name', 'away_team_name', 'league_name', 'is_cup', 'home_team_history_match_date_1', 'home_team_history_match_date_2', 'home_team_history_match_date_3', 'home_team_history_match_date_4', 'home_team_history_match_date_5', 'home_team_history_match_date_6', 'home_team_history_match_date_7', 'home_team_history_match_date_8', 'home_team_history_match_date_9', 'home_team_history_match_date_10', 'away_team_history_match_date_1', 'away_team_history_match_date_2', 'away_team_history_match_date_3', 'away_team_history_match_date_4', 'away_team_history_match_date_5', 'away_team_history_match_date_6', 'away_team_history_match_date_7', 'away_team_history_match_date_8', 'away_team_history_match_date_9', 'away_team_history_match_date_10']


## Replacing missing values in categorical features

In [7]:
# replacing missing values in categorical features
for feature in categorical_nan:
    train[feature] = train[feature].fillna('None')

In [8]:
train[categorical_nan].isna().sum()

home_team_name                     0
away_team_name                     0
league_name                        0
is_cup                             0
home_team_history_match_date_1     0
home_team_history_match_date_2     0
home_team_history_match_date_3     0
home_team_history_match_date_4     0
home_team_history_match_date_5     0
home_team_history_match_date_6     0
home_team_history_match_date_7     0
home_team_history_match_date_8     0
home_team_history_match_date_9     0
home_team_history_match_date_10    0
away_team_history_match_date_1     0
away_team_history_match_date_2     0
away_team_history_match_date_3     0
away_team_history_match_date_4     0
away_team_history_match_date_5     0
away_team_history_match_date_6     0
away_team_history_match_date_7     0
away_team_history_match_date_8     0
away_team_history_match_date_9     0
away_team_history_match_date_10    0
dtype: int64

## Handle numerical features with nan value

In [9]:
# Handle numerical features with nan value
numerical_nan = [feature for feature in train.columns if train[feature].isna().sum()>1 and train[feature].dtypes!='O']
print(numerical_nan)

['home_team_coach_id', 'away_team_coach_id', 'home_team_history_is_play_home_1', 'home_team_history_is_play_home_2', 'home_team_history_is_play_home_3', 'home_team_history_is_play_home_4', 'home_team_history_is_play_home_5', 'home_team_history_is_play_home_6', 'home_team_history_is_play_home_7', 'home_team_history_is_play_home_8', 'home_team_history_is_play_home_9', 'home_team_history_is_play_home_10', 'home_team_history_is_cup_1', 'home_team_history_is_cup_2', 'home_team_history_is_cup_3', 'home_team_history_is_cup_4', 'home_team_history_is_cup_5', 'home_team_history_is_cup_6', 'home_team_history_is_cup_7', 'home_team_history_is_cup_8', 'home_team_history_is_cup_9', 'home_team_history_is_cup_10', 'home_team_history_goal_1', 'home_team_history_goal_2', 'home_team_history_goal_3', 'home_team_history_goal_4', 'home_team_history_goal_5', 'home_team_history_goal_6', 'home_team_history_goal_7', 'home_team_history_goal_8', 'home_team_history_goal_9', 'home_team_history_goal_10', 'home_team_h

In [10]:
train[numerical_nan].isna().sum()

home_team_coach_id                  21721
away_team_coach_id                  21815
home_team_history_is_play_home_1     1159
home_team_history_is_play_home_2     2451
home_team_history_is_play_home_3     3773
                                    ...  
away_team_history_league_id_6        8426
away_team_history_league_id_7        9867
away_team_history_league_id_8       11295
away_team_history_league_id_9       12762
away_team_history_league_id_10      14216
Length: 162, dtype: int64

## Replacing the numerical Missing Values

In [11]:
for feature in numerical_nan:
    ## I will replace by using median since there are outliers
    median_value=train[feature].median()
    
    train[feature].fillna(median_value,inplace=True)
    
train[numerical_nan].isnull().sum()

home_team_coach_id                  0
away_team_coach_id                  0
home_team_history_is_play_home_1    0
home_team_history_is_play_home_2    0
home_team_history_is_play_home_3    0
                                   ..
away_team_history_league_id_6       0
away_team_history_league_id_7       0
away_team_history_league_id_8       0
away_team_history_league_id_9       0
away_team_history_league_id_10      0
Length: 162, dtype: int64

## Label Encoding

In [12]:
lbl= LabelEncoder()
y= train.iloc[:,-1:]
y= lbl.fit_transform(y)
y

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([324, 222, 217, ..., 691, 202, 194])

# Training

In [13]:
train, test = train_test_split( train, test_size=0.2, random_state=42,shuffle=True, stratify=None) #None is default
train.shape, test.shape

((88750, 190), (22188, 190))

In [14]:
from flaml import AutoML
automl = AutoML()

In [15]:
y = train.pop('target')
X = train

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42,shuffle=True, stratify=y)

In [17]:
automl.fit(X_train, y_train, task="classification",metric='log_loss',time_budget=900)

[flaml.automl.logger: 07-08 02:47:20] {1693} INFO - task = classification
[flaml.automl.logger: 07-08 02:47:20] {1700} INFO - Data split method: stratified
[flaml.automl.logger: 07-08 02:47:20] {1703} INFO - Evaluation method: holdout
[flaml.automl.logger: 07-08 02:47:20] {1801} INFO - Minimizing error metric: log_loss
[flaml.automl.logger: 07-08 02:47:21] {1911} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'catboost', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'lrl1']
[flaml.automl.logger: 07-08 02:47:21] {2221} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 07-08 02:47:22] {2347} INFO - Estimated sufficient time budget=67987s. Estimated necessary time budget=1669s.
[flaml.automl.logger: 07-08 02:47:22] {2394} INFO -  at 11.6s,	estimator lgbm's best error=1.0638,	best estimator lgbm's best error=1.0638
[flaml.automl.logger: 07-08 02:47:22] {2221} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 07-08 02:47:22] {2394} INFO -  at 12.4s,	estimato

/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[flaml.automl.logger: 07-08 02:47:25] {2394} INFO -  at 15.4s,	estimator xgboost's best error=1.0783,	best estimator lgbm's best error=1.0571
[flaml.automl.logger: 07-08 02:47:25] {2221} INFO - iteration 6, current learner lgbm
[flaml.automl.logger: 07-08 02:47:26] {2394} INFO -  at 16.2s,	estimator lgbm's best error=1.0504,	best estimator lgbm's best error=1.0504
[flaml.automl.logger: 07-08 02:47:26] {2221} INFO - iteration 7, current learner lgbm
[flaml.automl.logger: 07-08 02:47:27] {2394} INFO -  at 16.9s,	estimator lgbm's best error=1.0504,	best estimator lgbm's best error=1.0504
[flaml.automl.logger: 07-08 02:47:27] {2221} INFO - iteration 8, current learner xgboost


/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[flaml.automl.logger: 07-08 02:47:27] {2394} INFO -  at 17.3s,	estimator xgboost's best error=1.0783,	best estimator lgbm's best error=1.0504
[flaml.automl.logger: 07-08 02:47:27] {2221} INFO - iteration 9, current learner xgboost


...

[flaml.automl.logger: 07-08 03:01:08] {2394} INFO -  at 837.7s,	estimator xgb_limitdepth's best error=1.0264,	best estimator xgboost's best error=1.0168
[flaml.automl.logger: 07-08 03:01:08] {2221} INFO - iteration 100, current learner rf
[flaml.automl.logger: 07-08 03:01:13] {2394} INFO -  at 842.8s,	estimator rf's best error=1.0318,	best estimator xgboost's best error=1.0168
[flaml.automl.logger: 07-08 03:01:13] {2221} INFO - iteration 101, current learner rf
[flaml.automl.logger: 07-08 03:01:17] {2394} INFO -  at 847.1s,	estimator rf's best error=1.0318,	best estimator xgboost's best error=1.0168
[flaml.automl.logger: 07-08 03:01:17] {2221} INFO - iteration 102, current learner xgb_limitdepth


/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[flaml.automl.logger: 07-08 03:01:52] {2394} INFO -  at 881.9s,	estimator xgb_limitdepth's best error=1.0264,	best estimator xgboost's best error=1.0168
[flaml.automl.logger: 07-08 03:01:52] {2221} INFO - iteration 103, current learner rf
[flaml.automl.logger: 07-08 03:01:53] {2394} INFO -  at 882.7s,	estimator rf's best error=1.0318,	best estimator xgboost's best error=1.0168
[flaml.automl.logger: 07-08 03:01:53] {2221} INFO - iteration 104, current learner lgbm
[flaml.automl.logger: 07-08 03:01:58] {2394} INFO -  at 887.9s,	estimator lgbm's best error=1.0368,	best estimator xgboost's best error=1.0168
[flaml.automl.logger: 07-08 03:01:58] {2221} INFO - iteration 105, current learner rf
[flaml.automl.logger: 07-08 03:01:59] {2394} INFO -  at 889.0s,	estimator rf's best error=1.0318,	best estimator xgboost's best error=1.0168
[flaml.automl.logger: 07-08 03:01:59] {2221} INFO - iteration 106, current learner lgbm
[flaml.automl.logger: 07-08 03:02:05] {2394} INFO -  at 894.8s,	estimator 

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[flaml.automl.logger: 07-08 03:03:10] {2630} INFO - retrain xgboost for 58.8s
[flaml.automl.logger: 07-08 03:03:10] {2633} INFO - retrained model: XGBClassifier(base_score=None, booster=None, callbacks=[],
              colsample_bylevel=0.7089206179282082, colsample_bynode=None,
              colsample_bytree=0.6165783556569796, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy='lossguide',
              importance_type=None, interaction_constraints=None,
              learning_rate=0.11980646960410847, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=0, max_leaves=11,
              min_child_weight=36.9826704053053, missing=nan,
              monotone_constraints=None, n_estimators=396, n_jobs=-1,
              num_parallel_tree=None, objective='multi:softprob',
              predictor=None, ...)
[flaml.auto

# Result

In [18]:
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Best log_loss on validation data: {0:.4g}'.format(automl.best_loss))
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

Best ML leaner: xgboost
Best hyperparmeter config: {'n_estimators': 396, 'max_leaves': 11, 'min_child_weight': 36.9826704053053, 'learning_rate': 0.11980646960410847, 'subsample': 0.9048064340763577, 'colsample_bylevel': 0.7089206179282082, 'colsample_bytree': 0.6165783556569796, 'reg_alpha': 0.1289766547000545, 'reg_lambda': 0.019843909838391162}
Best log_loss on validation data: 1.017
Training duration of best run: 58.79 s
